# TCC - EDA - Análise 1
Este notebook é a uma continação do notebook TCC_PRE, o qual reprensenta especificamente a **primeira** análise feita sobre os dados.

# Análise 1

1. Selecionar todas as disciplinas obrigatorias do curso de Direito da UFRN, e mostrar em gráficos de barras a quantidade total de aprovações x reprovações em cada disciplina de um certo semestre, de acordo com a recomendação do SIGAA:

   - Cada descrição terá uma cor diferente, por exemplo, azul para aprovações e vermelho para reprovações.
   - Será feito um gráfico para cada conjunto de disciplinas obrigatórias de acordo com a sugestão do sigaa.
   - Haverá um gráfico detalhado, exibindo todos os tipos de aprovações e reprovações e um simplificado, unindo todos os tipos de aprovações em uma única categoria e algo similar para as reprovações.
   - Haverá também gráficos de barras com porcentagens relativas às quantidades de cada descrição.
   - Exibir qual foi a disciplina com maior quantidade de reprovações.
   - Exibir qual foi a disciplina com maior quantidade de trancamentos.


# Imports das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#Permitir que sejam exibidas inúmeras linhas e colunas no notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Carregamento do .csv contendo apenas as matrículas do curso de direito

Todos os datasets estão localizados no meu google drive, então é necessário configurar o colab de modo que seja possível realizar o acesso desses dados do meu google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Lendo o arquivo contendo as matrículas de direito.

In [ ]:
try:
  arquivo = f'/content/drive/MyDrive/UF/TCC/datasets/matriculas_direito.csv'
  matriculas_direito = pd.read_csv(arquivo, sep=';')
  print(f"Arquivo {arquivo} lido com sucesso.")
except FileNotFoundError:
  print(f"Arquivo {arquivo} não encontrado.")
except Exception as e:
  print(f"Ocorreu um erro ao ler o arquivo {arquivo}: {e}")


Arquivo /content/drive/MyDrive/UF/TCC/datasets/matriculas_direito.csv lido com sucesso.


In [ ]:
matriculas_direito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175500 entries, 0 to 175499
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   discente             175500 non-null  object 
 1   curso                175500 non-null  object 
 2   codigo_componente    175500 non-null  object 
 3   nome_componente      175500 non-null  object 
 4   media_final          172178 non-null  float64
 5   numero_total_faltas  172807 non-null  float64
 6   descricao            175500 non-null  object 
 7   semestre             175500 non-null  float64
dtypes: float64(3), object(5)
memory usage: 10.7+ MB


# Funcionalidade 1

## Definição de cores

In [ ]:
cores_grafico_barras_detalhado = {
  'APROVADO': '#3b6dc5',
  'APROVADO POR NOTA': '#a6beea',
  'REPROVADO': '#fadf70',
  'REPROVADO POR FALTAS': '#ffc12c',
  'REPROVADO POR MÉDIA E POR FALTAS': '#fdaa07',
  'REPROVADO POR NOTA E FALTA': '#f2700d',
  'REPROVADO POR NOTA': '#f0800d',
  'TRANCADO': '#ce0026'
}

cores_grafico_barras_simplificado = {
  'APROVADO': '#3b6dc5',
  'REPROVADO': '#f0800d',
  'TRANCADO': '#ce0026'
}

## Separação de disciplinas por semestre

In [ ]:
#p1 é o primeiro periodo, p2 o segundo, e assim sucessivamente
disciplinas_p1 = [
  'METODOLOGIA DA PESQUISA I',
  'CIENCIA POLITICA I',
  'SOCIOLOGIA E ANTROPOLOGIA GERAL',
  'INTRODUCAO AO ESTUDO DO DIREITO',
  'FILOSOFIA I'
]

disciplinas_p2 = [
  'DIREITO CIVIL I',
  'HISTORIA DO DIREITO',
  'DIREITO CONSTITUCIONAL I',
  'DIREITO PENAL I',
  'ECONOMIA POLITICA',
  'PSICOLOGIA APLICADA AO DIREITO'
]

disciplinas_p3 = [
  'DIREITO CIVIL II',
  'FILOSOFIA DO DIREITO',
  'TEORIA GERAL DO PROCESSO',
  'HERMENEUTICA JURIDICA E TEORIA DA ARGUMENTAcAO',
  'DIREITO CONSTITUCIONAL II',
  'DIREITO PENAL II'
]

disciplinas_p4 = [
  'DIREITO CIVIL III',
  'DIREITO EMPRESARIAL I',
  'DIREITO INTERNACIONAL PUBLICO',
  'SOCIOLOGIA JURIDICA',
  'DIREITO PENAL III',
  'DIREITO PROCESSUAL CIVIL I'
]

disciplinas_p5 = [
  'DIREITO CIVIL IV',
  'DIREITO EMPRESARIAL II',
  'DIREITO ADMINISTRATIVO I',
  'DIREITO PENAL IV',
  'DIREITO PROCESSUAL CIVIL II',
  'DIREITOS HUMANOS FUNDAMENTAIS'
]

disciplinas_p6 = [
  'DIREITO CIVIL V',
  'DIREITO EMPRESARIAL III',
  'DIREITO ADMINISTRATIVO II',
  'DIREITO PROCESSUAL CIVIL III',
  'DIREITO PROCESSUAL PENAL I',
  'DIREITO DAS RELACOES DE CONSUMO'
]

disciplinas_p7 = [
  'DIREITO CIVIL VI',
  'DIREITO DO TRABALHO I',
  #'PRÁTICA JURÍDICA I', #essa disciplina é um bloco formado pelas 4 disciplinas abaixo
  'AUTOCOMPOSICAO DE CONFLITOS: NEGOCIACAO, CONCILIACAO E MEDIACAO',
  'CARREIRAS JURIDICAS',
  'PECAS JURIDICAS II (EXTRAJUDICIAIS)',
  'PECAS JURIDICAS I (JUDICIAIS)',
  'DIREITO PROCESSUAL CIVIL IV',
  'DIREITO PROCESSUAL PENAL II',
  'ETICA GERAL E PROFISSIONAL'
]

disciplinas_p8 = [
  'METODOLOGIA DA PESQUISA II',
  'ORIENTACAO AO TCC I', #não aparece nos datasets de matrículas
  'DIREITO CIVIL VII',
  'DIREITO DO TRABALHO II',
  'DIREITO TRIBUTARIO',
  'DIREITO PROCESSUAL COLETIVO'
]

disciplinas_p9 = [
  'ORIENTACAO AO TCC II', #não aparece nos datasets de matrículas
  'DIREITO CIVIL VIII',
  'DIREITO DO TRABALHO III',
  'PRATICA JURIDICA III - ATENDIMENTOS',
  'DIREITO PROCESSUAL DO TRABALHO'
]

disciplinas_p10 = [
  'TRABALHO DE CONCLUSAO DE CURSO - TCC',
  #'PRÁTICA JURÍDICA IV' Essa disciplina tb não aparece, mas aparece apenas a abaixo:
  'ATENDIMENTOS - ANDAMENTOS PROCESSUAIS'
]

## Criação de dataframes auxiliares para cada bloco de disciplinas de um semestre

In [ ]:
#Dataframes contendo as disciplinas de um bloco de semestre específico
matriculas_p1 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p1)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p2 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p2)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p3 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p3)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p4 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p4)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p5 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p5)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p6 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p6)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p7 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p7)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p8 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p8)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p9 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p9)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')
matriculas_p10 = matriculas_direito[matriculas_direito['nome_componente'].isin(disciplinas_p10)].copy().groupby(['nome_componente', 'descricao']).size().reset_index(name='contagem')

Esses dataframes auxiliares armazenam a informação de quantos alunos foram aprovados/reprovados/trancaram uma determinada disciplina, como se pode ver abaixo:

In [ ]:
display(matriculas_p1)

,nome_componente,descricao,contagem
0,CIENCIA POLITICA I,APROVADO,2239
1,CIENCIA POLITICA I,APROVADO POR NOTA,187
2,CIENCIA POLITICA I,REPROVADO,86
3,CIENCIA POLITICA I,REPROVADO POR FALTAS,3
4,CIENCIA POLITICA I,REPROVADO POR MÉDIA E POR FALTAS,50
5,CIENCIA POLITICA I,REPROVADO POR NOTA,1
6,CIENCIA POLITICA I,REPROVADO POR NOTA E FALTA,1
7,CIENCIA POLITICA I,TRANCADO,61
8,FILOSOFIA I,APROVADO,2308
9,FILOSOFIA I,APROVADO POR NOTA,70


Para facilitar a geração do gráfico de barras, vou pivotar esse dataframe, transformando cada valor possível para descrição em uma coluna, e seus valores serão as contagens

In [ ]:
matriculas_p1_pivotada = matriculas_p1.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p2_pivotada = matriculas_p2.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p3_pivotada = matriculas_p3.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p4_pivotada = matriculas_p4.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p5_pivotada = matriculas_p5.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p6_pivotada = matriculas_p6.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p7_pivotada = matriculas_p7.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p8_pivotada = matriculas_p8.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p9_pivotada = matriculas_p9.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)
matriculas_p10_pivotada = matriculas_p10.pivot(index='nome_componente', columns='descricao', values='contagem').fillna(0)

A estrutura do novo dataframe pode ser vista abaixo:

In [ ]:
matriculas_p1_pivotada

descricao,APROVADO,APROVADO POR NOTA,REPROVADO,REPROVADO POR FALTAS,REPROVADO POR MÉDIA E POR FALTAS,REPROVADO POR NOTA,REPROVADO POR NOTA E FALTA,TRANCADO
nome_componente,,,,,,,,
CIENCIA POLITICA I,2239.0,187.0,86.0,3.0,50.0,1.0,1.0,61.0
FILOSOFIA I,2308.0,70.0,92.0,1.0,51.0,6.0,0.0,66.0
INTRODUCAO AO ESTUDO DO DIREITO,2262.0,242.0,41.0,0.0,107.0,3.0,3.0,53.0
METODOLOGIA DA PESQUISA I,2302.0,34.0,64.0,4.0,50.0,0.0,0.0,44.0
SOCIOLOGIA E ANTROPOLOGIA GERAL,2250.0,129.0,45.0,0.0,78.0,0.0,1.0,62.0


## Gráficos de aprovações e reprovações em disciplinas do primeiro semestre

In [ ]:
grafico_barras_p1 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p1_pivotada.index,
    y=matriculas_p1_pivotada[col],
    text=matriculas_p1_pivotada[col],
    textposition='outside',
    marker_color=cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p1_pivotada.columns
])

grafico_barras_p1.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do primeiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height= 600
)

grafico_barras_p1.show()

In [ ]:
#Converto as quantidades em percentuais
matriculas_p1_pivotada_percentual = matriculas_p1_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p1_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p1_pivotada_percentual.index,
    y=matriculas_p1_pivotada_percentual[col],
    text=matriculas_p1_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p1_pivotada_percentual.columns
])

grafico_barras_p1_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do primeiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p1_percentual.show()

### Gráfico simplificado de aprovações e reprovações em disciplinas do primeiro semestre

In [ ]:
matriculas_p1_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p1_pivotada['APROVADO'] + matriculas_p1_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p1_pivotada['REPROVADO'] + matriculas_p1_pivotada['REPROVADO POR FALTAS'] + matriculas_p1_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p1_pivotada['REPROVADO POR NOTA'] + matriculas_p1_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p1_pivotada['TRANCADO']
})

display(matriculas_p1_pivotada_simplificada)

,APROVADO,REPROVADO,TRANCADO
nome_componente,,,
CIENCIA POLITICA I,2426.0,141.0,61.0
FILOSOFIA I,2378.0,150.0,66.0
INTRODUCAO AO ESTUDO DO DIREITO,2504.0,154.0,53.0
METODOLOGIA DA PESQUISA I,2336.0,118.0,44.0
SOCIOLOGIA E ANTROPOLOGIA GERAL,2379.0,124.0,62.0


In [ ]:
grafico_barras_p1_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p1_pivotada_simplificada.index,
    y=matriculas_p1_pivotada_simplificada[col],
    text=matriculas_p1_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p1_pivotada_simplificada.columns
])

grafico_barras_p1_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do primeiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p1_simplificado.show()

In [ ]:
matriculas_p1_pivotada_simplificada_percentual = matriculas_p1_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p1_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p1_pivotada_simplificada_percentual.index,
    y=matriculas_p1_pivotada_simplificada_percentual[col],
    text=matriculas_p1_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p1_pivotada_simplificada_percentual.columns
])

grafico_barras_p1_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do primeiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p1_simplificado_percentual.show()

### Disciplina com maior quantidade de aprovações do primeiro semestre

In [ ]:
disciplina_mais_aprovada_p1 = matriculas_p1_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovações = matriculas_p1_pivotada_simplificada.loc[disciplina_mais_aprovada_p1, 'APROVADO']

print(f"A disciplina com mais aprovações no primeiro semestre é: {disciplina_mais_aprovada_p1} com {int(contagem_aprovações)} aprovações.")

A disciplina com mais aprovações no primeiro semestre é: INTRODUCAO AO ESTUDO DO DIREITO com 2504 aprovações.


### Disciplina com maior quantidade de reprovações do primeiro semestre

In [ ]:
disciplina_mais_reprovada_p1 = matriculas_p1_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p1_pivotada_simplificada.loc[disciplina_mais_reprovada_p1, 'REPROVADO']

print(f"A disciplina com mais reprovações no primeiro semestre é: {disciplina_mais_reprovada_p1} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no primeiro semestre é: INTRODUCAO AO ESTUDO DO DIREITO com 154 reprovações.


### Disciplina com maior quantidade de trancamentos do primeiro semestre

In [ ]:
disciplina_mais_trancada_p1 = matriculas_p1_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p1_pivotada_simplificada.loc[disciplina_mais_trancada_p1, 'TRANCADO']

print(f"A disciplina com mais trancamentos no primeiro semestre é: {disciplina_mais_trancada_p1} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no primeiro semestre é: FILOSOFIA I com 66 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do segundo semestre

In [ ]:
grafico_barras_p2 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p2_pivotada.index,
    y=matriculas_p2_pivotada[col],
    text=matriculas_p2_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026'),
  ) for col in matriculas_p1_pivotada.columns
])

grafico_barras_p2.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do segundo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600
)

grafico_barras_p2.show()

In [ ]:
matriculas_p2_pivotada_percentual = matriculas_p2_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p2_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p2_pivotada_percentual.index,
    y=matriculas_p2_pivotada_percentual[col],
    text=matriculas_p2_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p2_pivotada_percentual.columns
])

grafico_barras_p2_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações em disciplinas obrigatórias do segundo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p2_percentual.show()

In [ ]:
matriculas_p2_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p2_pivotada['APROVADO'] + matriculas_p2_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p2_pivotada['REPROVADO'] + matriculas_p2_pivotada['REPROVADO POR FALTAS'] + matriculas_p2_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p2_pivotada['REPROVADO POR NOTA'] + matriculas_p2_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p2_pivotada['TRANCADO']
})

grafico_barras_p2_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p2_pivotada_simplificada.index,
    y=matriculas_p2_pivotada_simplificada[col],
    text=matriculas_p2_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p2_pivotada_simplificada.columns
])

grafico_barras_p2_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do segundo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p2_simplificado.show()

In [ ]:
matriculas_p2_pivotada_simplificada_percentual = matriculas_p2_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p2_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p2_pivotada_simplificada_percentual.index,
    y=matriculas_p2_pivotada_simplificada_percentual[col],
    text=matriculas_p2_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p2_pivotada_simplificada_percentual.columns
])

grafico_barras_p2_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do segundo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p2_simplificado_percentual.show()

Disciplina com maior quantidade de aprovações no segundo semestre

In [ ]:
disciplina_mais_aprovada_p2 = matriculas_p2_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p2_pivotada_simplificada.loc[disciplina_mais_aprovada_p2, 'APROVADO']

print(f"A disciplina com mais aprovações no segundo semestre é: {disciplina_mais_aprovada_p2} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no segundo semestre é: DIREITO CIVIL I com 2587 aprovações.


### Disciplina com maior quantidade de reprovações no segundo semestre

In [ ]:
disciplina_mais_reprovada_p2 = matriculas_p2_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p2_pivotada_simplificada.loc[disciplina_mais_reprovada_p2, 'REPROVADO']

print(f"A disciplina com mais reprovações no segundo semestre é: {disciplina_mais_reprovada_p2} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no segundo semestre é: DIREITO PENAL I com 113 reprovações.


### Disciplina com maior quantidade de trancamentos no segundo semestre

In [ ]:
disciplina_mais_trancada_p2 = matriculas_p2_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p2_pivotada_simplificada.loc[disciplina_mais_trancada_p2, 'TRANCADO']

print(f"A disciplina com mais trancamentos no segundo semestre é: {disciplina_mais_trancada_p2} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no segundo semestre é: ECONOMIA POLITICA com 43 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do terceiro semestre

In [ ]:
grafico_barras_p3 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p3_pivotada.index,
    y=matriculas_p3_pivotada[col],
    text=matriculas_p3_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p3_pivotada.columns
])

grafico_barras_p3.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do terceiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p3.show()

In [ ]:
matriculas_p3_pivotada_percentual = matriculas_p3_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p3_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p3_pivotada_percentual.index,
    y=matriculas_p3_pivotada_percentual[col],
    text=matriculas_p3_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p3_pivotada_percentual.columns
])

grafico_barras_p3_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do terceiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p3_percentual.show()

In [ ]:
matriculas_p3_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p3_pivotada['APROVADO'] + matriculas_p3_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p3_pivotada['REPROVADO'] + matriculas_p3_pivotada['REPROVADO POR FALTAS'] + matriculas_p3_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p3_pivotada['REPROVADO POR NOTA'] + matriculas_p3_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p3_pivotada['TRANCADO']
})

grafico_barras_p3_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p3_pivotada_simplificada.index,
    y=matriculas_p3_pivotada_simplificada[col],
    text=matriculas_p3_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p3_pivotada_simplificada.columns
])

grafico_barras_p3_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do terceiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p3_simplificado.show()

In [ ]:
matriculas_p3_pivotada_simplificada_percentual = matriculas_p3_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p3_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p3_pivotada_simplificada_percentual.index,
    y=matriculas_p3_pivotada_simplificada_percentual[col],
    text=matriculas_p3_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p3_pivotada_simplificada_percentual.columns
])

grafico_barras_p3_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do terceiro semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p3_simplificado_percentual.show()

### Disciplina com maior quantidade de aprovações no terceiro semestre

In [ ]:
disciplina_mais_arovada_p3 = matriculas_p3_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p3_pivotada_simplificada.loc[disciplina_mais_arovada_p3, 'APROVADO']

print(f"A disciplina com mais aprovações no terceiro semestre é: {disciplina_mais_arovada_p3} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no terceiro semestre é: DIREITO CONSTITUCIONAL II com 2481 aprovações.


### Disciplina com maior quantidade de reprovações no terceiro semestre

In [ ]:
disciplina_mais_reprovada_p3 = matriculas_p3_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p3_pivotada_simplificada.loc[disciplina_mais_reprovada_p3, 'REPROVADO']

print(f"A disciplina com mais reprovações no terceiro semestre é: {disciplina_mais_reprovada_p3} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no terceiro semestre é: DIREITO PENAL II com 148 reprovações.


### Disciplina com maior quantidade de trancamentos no terceiro semestre

In [ ]:
disciplina_mais_trancada_p3 = matriculas_p3_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p3_pivotada_simplificada.loc[disciplina_mais_trancada_p3, 'TRANCADO']

print(f"A disciplina com mais trancamentos no terceiro semestre é: {disciplina_mais_trancada_p3} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no terceiro semestre é: FILOSOFIA DO DIREITO com 43 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do quarto semestre

In [ ]:
grafico_barras_p4 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p4_pivotada.index,
    y=matriculas_p4_pivotada[col],
    text=matriculas_p4_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p4_pivotada.columns
])

grafico_barras_p4.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do quarto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p4.show()

In [ ]:
matriculas_p4_pivotada_percentual = matriculas_p4_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p4_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p4_pivotada_percentual.index,
    y=matriculas_p4_pivotada_percentual[col],
    text=matriculas_p4_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p4_pivotada_percentual.columns
])

grafico_barras_p4_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do quarto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p4_percentual.show()

In [ ]:
matriculas_p4_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p4_pivotada['APROVADO'] + matriculas_p4_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p4_pivotada['REPROVADO'] + matriculas_p4_pivotada['REPROVADO POR FALTAS'] + matriculas_p4_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p4_pivotada['REPROVADO POR NOTA'] + matriculas_p4_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p4_pivotada['TRANCADO']
})

grafico_barras_p4_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p4_pivotada_simplificada.index,
    y=matriculas_p4_pivotada_simplificada[col],
    text=matriculas_p4_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p4_pivotada_simplificada.columns
])

grafico_barras_p4_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do quarto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p4_simplificado.show()

In [ ]:
matriculas_p4_pivotada_simplificada_percentual = matriculas_p4_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p4_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p4_pivotada_simplificada_percentual.index,
    y=matriculas_p4_pivotada_simplificada_percentual[col],
    text=matriculas_p4_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p4_pivotada_simplificada_percentual.columns
])

grafico_barras_p4_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do quarto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p4_simplificado_percentual.show()

### Disciplina com maior quantidade de aprovações no quarto semestre

In [ ]:
disciplina_mais_aprovada_p4 = matriculas_p4_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p4_pivotada_simplificada.loc[disciplina_mais_aprovada_p4, 'APROVADO']

print(f"A disciplina com mais aprovações no quarto semestre é: {disciplina_mais_aprovada_p4} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no quarto semestre é: DIREITO CIVIL III com 2463 aprovações.


### Disciplina com maior quantidade de reprovações no quarto semestre

In [ ]:
disciplina_mais_reprovada_p4 = matriculas_p4_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p4_pivotada_simplificada.loc[disciplina_mais_reprovada_p4, 'REPROVADO']

print(f"A disciplina com mais reprovações no quarto semestre é: {disciplina_mais_reprovada_p4} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no quarto semestre é: DIREITO INTERNACIONAL PUBLICO com 340 reprovações.


### Disciplina com maior quantidade de trancamentos no quarto semestre

In [ ]:
disciplina_mais_trancada_p4 = matriculas_p4_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p4_pivotada_simplificada.loc[disciplina_mais_trancada_p4, 'TRANCADO']

print(f"A disciplina com mais trancamentos no quarto semestre é: {disciplina_mais_trancada_p4} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no quarto semestre é: DIREITO INTERNACIONAL PUBLICO com 86 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do quinto semestre

In [ ]:
grafico_barras_p5 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p5_pivotada.index,
    y=matriculas_p5_pivotada[col],
    text=matriculas_p5_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p5_pivotada.columns
])

grafico_barras_p5.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do quinto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p5.show()

In [ ]:
matriculas_p5_pivotada_percentual = matriculas_p5_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p5_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p5_pivotada_percentual.index,
    y=matriculas_p5_pivotada_percentual[col],
    text=matriculas_p5_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p5_pivotada_percentual.columns
])

grafico_barras_p5_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do quinto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p5_percentual.show()

In [ ]:
matriculas_p5_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p5_pivotada['APROVADO'] + matriculas_p5_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p5_pivotada['REPROVADO'] + matriculas_p5_pivotada['REPROVADO POR FALTAS'] + matriculas_p5_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p5_pivotada['REPROVADO POR NOTA'] + matriculas_p5_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p5_pivotada['TRANCADO']
})

grafico_barras_p5_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p5_pivotada_simplificada.index,
    y=matriculas_p5_pivotada_simplificada[col],
    text=matriculas_p5_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p5_pivotada_simplificada.columns
])

grafico_barras_p5_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do quinto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p5_simplificado.show()

In [ ]:
matriculas_p5_pivotada_simplificada_percentual = matriculas_p5_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p5_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p5_pivotada_simplificada_percentual.index,
    y=matriculas_p5_pivotada_simplificada_percentual[col],
    text=matriculas_p5_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p5_pivotada_simplificada_percentual.columns
])

grafico_barras_p5_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do quinto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p5_simplificado_percentual.show()

### Disciplina com maior quantidade de aprovações no quinto semestre

In [ ]:
disciplina_mais_aprovada_p5 = matriculas_p5_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p5_pivotada_simplificada.loc[disciplina_mais_aprovada_p5, 'APROVADO']

print(f"A disciplina com mais aprovações no quinto semestre é: {disciplina_mais_aprovada_p5} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no quinto semestre é: DIREITO CIVIL IV com 2538 aprovações.


### Disciplina com maior quantidade de reprovações no quinto semestre

In [ ]:
disciplina_mais_reprovada_p5 = matriculas_p5_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p5_pivotada_simplificada.loc[disciplina_mais_reprovada_p5, 'REPROVADO']

print(f"A disciplina com mais reprovações no quinto semestre é: {disciplina_mais_reprovada_p5} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no quinto semestre é: DIREITO EMPRESARIAL II com 99 reprovações.


### Disciplina com maior quantidade de trancamentos no quinto semestre

In [ ]:
disciplina_mais_trancada_p5 = matriculas_p5_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p5_pivotada_simplificada.loc[disciplina_mais_trancada_p5, 'TRANCADO']

print(f"A disciplina com mais trancamentos no quinto semestre é: {disciplina_mais_trancada_p5} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no quinto semestre é: DIREITOS HUMANOS FUNDAMENTAIS com 52 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do sexto semestre

In [ ]:
grafico_barras_p6 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p6_pivotada.index,
    y=matriculas_p6_pivotada[col],
    text=matriculas_p6_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p6_pivotada.columns
])

grafico_barras_p6.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do sexto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p6.show()

In [ ]:
matriculas_p6_pivotada_percentual = matriculas_p6_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p6_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p6_pivotada_percentual.index,
    y=matriculas_p6_pivotada_percentual[col],
    text=matriculas_p6_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p6_pivotada_percentual.columns
])

grafico_barras_p6_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do sexto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p6_percentual.show()

In [ ]:
matriculas_p6_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p6_pivotada['APROVADO'] + matriculas_p6_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p6_pivotada['REPROVADO'] + matriculas_p6_pivotada['REPROVADO POR FALTAS'] + matriculas_p6_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p6_pivotada['REPROVADO POR NOTA'] + matriculas_p6_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p6_pivotada['TRANCADO']
})

grafico_barras_p6_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p6_pivotada_simplificada.index,
    y=matriculas_p6_pivotada_simplificada[col],
    text=matriculas_p6_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p6_pivotada_simplificada.columns
])

grafico_barras_p6_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do sexto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p6_simplificado.show()

In [ ]:
matriculas_p6_pivotada_simplificada_percentual = matriculas_p6_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p6_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p6_pivotada_simplificada_percentual.index,
    y=matriculas_p6_pivotada_simplificada_percentual[col],
    text=matriculas_p6_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p6_pivotada_simplificada_percentual.columns
])

grafico_barras_p6_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do sexto semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p6_simplificado_percentual.show()

### Disciplina com maior quantidade de aprovações no sexto semestre

In [ ]:
disciplina_mais_aprovada_p6 = matriculas_p6_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p6_pivotada_simplificada.loc[disciplina_mais_aprovada_p6, 'APROVADO']

print(f"A disciplina com mais aprovações no sexto semestre é: {disciplina_mais_aprovada_p6} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no sexto semestre é: DIREITO DAS RELACOES DE CONSUMO com 2480 aprovações.


### Disciplina com maior quantidade de reprovações no sexto semestre

In [ ]:
disciplina_mais_reprovada_p6 = matriculas_p6_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p6_pivotada_simplificada.loc[disciplina_mais_reprovada_p6, 'REPROVADO']

print(f"A disciplina com mais reprovações no sexto semestre é: {disciplina_mais_reprovada_p6} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no sexto semestre é: DIREITO PROCESSUAL PENAL I com 102 reprovações.


### Disciplina com maior quantidade de trancamentos no sexto semestre

In [ ]:
disciplina_mais_trancada_p6 = matriculas_p6_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p6_pivotada_simplificada.loc[disciplina_mais_trancada_p6, 'TRANCADO']

print(f"A disciplina com mais trancamentos no sexto semestre é: {disciplina_mais_trancada_p6} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no sexto semestre é: DIREITO PROCESSUAL PENAL I com 52 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do sétimo semestre

In [ ]:
grafico_barras_p7 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p7_pivotada.index,
    y=matriculas_p7_pivotada[col],
    text=matriculas_p7_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p7_pivotada.columns
])

grafico_barras_p7.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do sétimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=700,
  #width = 3500
)

grafico_barras_p7.show()

In [ ]:
matriculas_p7_pivotada_percentual = matriculas_p7_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p7_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p7_pivotada_percentual.index,
    y=matriculas_p7_pivotada_percentual[col],
    text=matriculas_p7_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p7_pivotada_percentual.columns
])

grafico_barras_p7_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do sétimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600
)

grafico_barras_p7_percentual.show()

In [ ]:
matriculas_p7_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p7_pivotada['APROVADO'] + matriculas_p7_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p7_pivotada['REPROVADO'] + matriculas_p7_pivotada['REPROVADO POR FALTAS'] + matriculas_p7_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p7_pivotada['REPROVADO POR NOTA'] + matriculas_p7_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p7_pivotada['TRANCADO']
})

grafico_barras_p7_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p7_pivotada_simplificada.index,
    y=matriculas_p7_pivotada_simplificada[col],
    text=matriculas_p7_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p7_pivotada_simplificada.columns
])

grafico_barras_p7_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do sétimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p7_simplificado.show()

In [ ]:
matriculas_p7_pivotada_simplificada_percentual = matriculas_p7_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p7_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p7_pivotada_simplificada_percentual.index,
    y=matriculas_p7_pivotada_simplificada_percentual[col],
    text=matriculas_p7_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p7_pivotada_simplificada_percentual.columns
])

grafico_barras_p7_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do sétimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p7_simplificado_percentual.show()

In [ ]:
disciplina_mais_aprovada_p7 = matriculas_p7_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p7_pivotada_simplificada.loc[disciplina_mais_aprovada_p7, 'APROVADO']

print(f"A disciplina com mais aprovações no sétimo semestre é: {disciplina_mais_aprovada_p7} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no sétimo semestre é: ETICA GERAL E PROFISSIONAL com 2478 aprovações.


### Disciplina com maior quantidade de reprovações no sétimo semestre

In [ ]:
disciplina_mais_reprovada_p7 = matriculas_p7_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p7_pivotada_simplificada.loc[disciplina_mais_reprovada_p7, 'REPROVADO']

print(f"A disciplina com mais reprovações no sétimo semestre é: {disciplina_mais_reprovada_p7} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no sétimo semestre é: DIREITO CIVIL VI com 92 reprovações.


### Disciplina com maior quantidade de trancamentos no sétimo semestre

In [ ]:
disciplina_mais_trancada_p7 = matriculas_p7_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p7_pivotada_simplificada.loc[disciplina_mais_trancada_p7, 'TRANCADO']

print(f"A disciplina com mais trancamentos no sétimo semestre é: {disciplina_mais_trancada_p7} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no sétimo semestre é: ETICA GERAL E PROFISSIONAL com 39 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do oitavo semestre

In [ ]:
grafico_barras_p8 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p8_pivotada.index,
    y=matriculas_p8_pivotada[col],
    text=matriculas_p8_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p8_pivotada.columns
])

grafico_barras_p8.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do oitavo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p8.show()

In [ ]:
matriculas_p8_pivotada_percentual = matriculas_p8_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p8_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p8_pivotada_percentual.index,
    y=matriculas_p8_pivotada_percentual[col],
    text=matriculas_p8_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p8_pivotada_percentual.columns
])

grafico_barras_p8_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do oitavo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p8_percentual.show()

In [ ]:
matriculas_p8_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p8_pivotada['APROVADO'] + matriculas_p8_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p8_pivotada['REPROVADO'] + matriculas_p8_pivotada['REPROVADO POR FALTAS'] + matriculas_p8_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p8_pivotada['REPROVADO POR NOTA'] + matriculas_p8_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p8_pivotada['TRANCADO']
})

grafico_barras_p8_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p8_pivotada_simplificada.index,
    y=matriculas_p8_pivotada_simplificada[col],
    text=matriculas_p8_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p8_pivotada_simplificada.columns
])

grafico_barras_p8_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do oitavo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p8_simplificado.show()

In [ ]:
matriculas_p8_pivotada_simplificada_percentual = matriculas_p8_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p8_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p8_pivotada_simplificada_percentual.index,
    y=matriculas_p8_pivotada_simplificada_percentual[col],
    text=matriculas_p8_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p8_pivotada_simplificada_percentual.columns
])

grafico_barras_p8_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do oitavo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p8_simplificado_percentual.show()

### Disciplina com maior número de aprovações no oitavo semestre

In [ ]:
disciplina_mais_aprovada_p8 = matriculas_p8_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p8_pivotada_simplificada.loc[disciplina_mais_aprovada_p8, 'APROVADO']

print(f"A disciplina com mais aprovações no oitavo semestre é: {disciplina_mais_aprovada_p8} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no oitavo semestre é: DIREITO CIVIL VII com 2434 aprovações.


### Disciplina com maior número de reprovações no oitavo semestre



In [ ]:
disciplina_mais_reprovada_p8 = matriculas_p8_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p8_pivotada_simplificada.loc[disciplina_mais_reprovada_p8, 'REPROVADO']

print(f"A disciplina com mais reprovações no oitavo semestre é: {disciplina_mais_reprovada_p8} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no oitavo semestre é: METODOLOGIA DA PESQUISA II com 149 reprovações.


### Disciplina com maior número de trancamentos no oitavo semestre



In [ ]:
disciplina_mais_trancada_p8 = matriculas_p8_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamentos = matriculas_p8_pivotada_simplificada.loc[disciplina_mais_trancada_p8, 'TRANCADO']

print(f"A disciplina com mais trancamentos no oitavo semestre é: {disciplina_mais_trancada_p8} com {int(contagem_trancamentos)} trancamentos.")

A disciplina com mais trancamentos no oitavo semestre é: METODOLOGIA DA PESQUISA II com 77 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do nono semestre

In [ ]:
grafico_barras_p9 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p9_pivotada.index,
    y=matriculas_p9_pivotada[col],
    text=matriculas_p9_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p9_pivotada.columns
])

grafico_barras_p9.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos em disciplinas obrigatórias do nono semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p9.show()

In [ ]:
matriculas_p8_pivotada_percentual = matriculas_p8_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p8_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p8_pivotada_percentual.index,
    y=matriculas_p8_pivotada_percentual[col],
    text=matriculas_p8_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p8_pivotada_percentual.columns
])

grafico_barras_p8_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do nono semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p8_percentual.show()

In [ ]:
matriculas_p9_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p9_pivotada['APROVADO'] + matriculas_p9_pivotada['APROVADO POR NOTA'],
    'REPROVADO': matriculas_p9_pivotada['REPROVADO'] + matriculas_p9_pivotada['REPROVADO POR FALTAS'] + matriculas_p9_pivotada['REPROVADO POR MÉDIA E POR FALTAS'] + matriculas_p9_pivotada['REPROVADO POR NOTA'] + matriculas_p9_pivotada['REPROVADO POR NOTA E FALTA'],
    'TRANCADO': matriculas_p9_pivotada['TRANCADO']
})

grafico_barras_p9_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p9_pivotada_simplificada.index,
    y=matriculas_p9_pivotada_simplificada[col],
    text=matriculas_p9_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p9_pivotada_simplificada.columns
])

grafico_barras_p9_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do nono semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p9_simplificado.show()

In [ ]:
matriculas_p9_pivotada_simplificada_percentual = matriculas_p9_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p9_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p9_pivotada_simplificada_percentual.index,
    y=matriculas_p9_pivotada_simplificada_percentual[col],
    text=matriculas_p9_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p9_pivotada_simplificada_percentual.columns
])

grafico_barras_p9_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do nono semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600,
)

grafico_barras_p9_simplificado_percentual.show()

### Disciplina com maior número de aprovações no nono semestre

In [ ]:
disciplina_mais_aprovada_p9 = matriculas_p9_pivotada_simplificada['APROVADO'].idxmax()
contagem_aprovacoes = matriculas_p9_pivotada_simplificada.loc[disciplina_mais_aprovada_p9, 'APROVADO']

print(f"A disciplina com mais aprovações no nono semestre é: {disciplina_mais_aprovada_p9} com {int(contagem_aprovacoes)} aprovações.")

A disciplina com mais aprovações no nono semestre é: PRATICA JURIDICA III - ATENDIMENTOS com 2377 aprovações.


### Disciplina com maior número de reprovações no nono semestre

In [ ]:
disciplina_mais_reprovada_p9 = matriculas_p9_pivotada_simplificada['REPROVADO'].idxmax()
contagem_reprovacoes = matriculas_p9_pivotada_simplificada.loc[disciplina_mais_reprovada_p9, 'REPROVADO']

print(f"A disciplina com mais reprovações no nono semestre é: {disciplina_mais_reprovada_p9} com {int(contagem_reprovacoes)} reprovações.")

A disciplina com mais reprovações no nono semestre é: DIREITO PROCESSUAL DO TRABALHO com 117 reprovações.


### Disciplina com maior número de trancamentos no nono semestre

In [ ]:
disciplina_mais_trancada_p9 = matriculas_p9_pivotada_simplificada['TRANCADO'].idxmax()
contagem_trancamento = matriculas_p9_pivotada_simplificada.loc[disciplina_mais_trancada_p9, 'TRANCADO']

print(f"A disciplina com mais trancamentos no nono semestre é: {disciplina_mais_trancada_p9} com {int(contagem_trancamento)} trancamentos.")

A disciplina com mais trancamentos no nono semestre é: DIREITO PROCESSUAL DO TRABALHO com 66 trancamentos.


## Gráficos de aprovações e reprovações em disciplinas do décimo semestre

In [ ]:
grafico_barras_p10 = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p10_pivotada.index,
    y=matriculas_p10_pivotada[col],
    text=matriculas_p10_pivotada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p10_pivotada.columns
])

grafico_barras_p10.update_layout(
  title='Contagem de aprovações, reprovações e trancamentos na disciplina ATENDIMENTOS - ANDAMENTOS PROCESSUAIS, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de discentes',
  barmode='group',
  height=900,
)

grafico_barras_p10.show()

In [ ]:
matriculas_p10_pivotada_percentual = matriculas_p10_pivotada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p10_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p10_pivotada_percentual.index,
    y=matriculas_p10_pivotada_percentual[col],
    text=matriculas_p10_pivotada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_detalhado.get(col, '#ce0026')
  ) for col in matriculas_p10_pivotada_percentual.columns
])

grafico_barras_p10_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (detalhada) em disciplinas obrigatórias do décimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Porcentagem de discentes',
  barmode='group',
  height=600,
)

grafico_barras_p10_percentual.show()

In [ ]:
matriculas_p10_pivotada_simplificada = pd.DataFrame({
    'APROVADO': matriculas_p10_pivotada['APROVADO'],
    'REPROVADO': matriculas_p10_pivotada['REPROVADO'] + matriculas_p10_pivotada['REPROVADO POR FALTAS'] + matriculas_p10_pivotada['REPROVADO POR MÉDIA E POR FALTAS'],
    'TRANCADO': matriculas_p10_pivotada['TRANCADO']
})

grafico_barras_p10_simplificado = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p10_pivotada_simplificada.index,
    y=matriculas_p10_pivotada_simplificada[col],
    text=matriculas_p10_pivotada_simplificada[col],
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p10_pivotada_simplificada.columns
])

grafico_barras_p10_simplificado.update_layout(
  title='Contagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do décimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=600
)

grafico_barras_p10_simplificado.show()

In [ ]:
matriculas_p10_pivotada_simplificada_percentual = matriculas_p10_pivotada_simplificada.apply(lambda x: x / x.sum() * 100, axis=1)
grafico_barras_p10_simplificado_percentual = go.Figure(data=[
  go.Bar(
    name=col,
    x=matriculas_p10_pivotada_simplificada_percentual.index,
    y=matriculas_p10_pivotada_simplificada_percentual[col],
    text=matriculas_p10_pivotada_simplificada_percentual[col].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color = cores_grafico_barras_simplificado.get(col, '#ce0026')
  ) for col in matriculas_p10_pivotada_simplificada_percentual.columns
])

grafico_barras_p10_simplificado_percentual.update_layout(
  title='Porcentagem de aprovações e reprovações (simplificada) em disciplinas obrigatórias do décimo semestre do curso de Direito, entre 2011.1 e 2024.2',
  xaxis_title='Disciplina',
  yaxis_title='Número de Alunos',
  barmode='group',
  height=800,
)

grafico_barras_p10_simplificado_percentual.show()

In [ ]:
matriculas_simplificada_total = pd.concat([
    matriculas_p1_pivotada_simplificada,
    matriculas_p2_pivotada_simplificada,
    matriculas_p3_pivotada_simplificada,
    matriculas_p4_pivotada_simplificada,
    matriculas_p5_pivotada_simplificada,
    matriculas_p6_pivotada_simplificada,
    matriculas_p7_pivotada_simplificada,
    matriculas_p8_pivotada_simplificada,
    matriculas_p9_pivotada_simplificada,
    matriculas_p10_pivotada_simplificada
])

ranking_disciplinas = matriculas_simplificada_total.groupby('nome_componente')[['REPROVADO', 'TRANCADO']].sum().reset_index()
ranking_reprovacoes = ranking_disciplinas.sort_values(by='REPROVADO', ascending=False).head(10)
ranking_trancamentos = ranking_disciplinas.sort_values(by='TRANCADO', ascending=False).head(10)

print("Top 10 disciplinas com maior quantidade de reprovações:")
display(ranking_reprovacoes)

print("\nTop 10 disciplinas com maior quantidade de trancamentos:")
display(ranking_trancamentos)

Top 10 disciplinas com maior quantidade de reprovações:


,nome_componente,REPROVADO,TRANCADO
23,DIREITO INTERNACIONAL PUBLICO,340.0,86.0
43,INTRODUCAO AO ESTUDO DO DIREITO,154.0,53.0
41,FILOSOFIA I,150.0,66.0
45,METODOLOGIA DA PESQUISA II,149.0,77.0
25,DIREITO PENAL II,148.0,33.0
3,CIENCIA POLITICA I,141.0,61.0
50,SOCIOLOGIA E ANTROPOLOGIA GERAL,124.0,62.0
44,METODOLOGIA DA PESQUISA I,118.0,44.0
33,DIREITO PROCESSUAL DO TRABALHO,117.0,66.0
24,DIREITO PENAL I,113.0,37.0



Top 10 disciplinas com maior quantidade de trancamentos:


,nome_componente,REPROVADO,TRANCADO
23,DIREITO INTERNACIONAL PUBLICO,340.0,86.0
45,METODOLOGIA DA PESQUISA II,149.0,77.0
41,FILOSOFIA I,150.0,66.0
33,DIREITO PROCESSUAL DO TRABALHO,117.0,66.0
50,SOCIOLOGIA E ANTROPOLOGIA GERAL,124.0,62.0
3,CIENCIA POLITICA I,141.0,61.0
36,DIREITO TRIBUTARIO,78.0,57.0
43,INTRODUCAO AO ESTUDO DO DIREITO,154.0,53.0
34,DIREITO PROCESSUAL PENAL I,102.0,52.0
37,DIREITOS HUMANOS FUNDAMENTAIS,94.0,52.0
